In [2]:
require(data.table)
require(tidyverse)

## 1. Read data 

In [3]:
dat.mat <- fread('../dat/1901/alpha_beta.promoter.long_matrix_w_transcripts.txt')
dim(dat.mat)
head(dat.mat)
#dat.mat$cell <- NULL #no need cell id 

[1] 19412387        7

seq,start,end,gene,cluster,cell,transcript.idx
chr1,859052,860562,SAMD11,alpha_1,Islet1-fresh_AGACACCTAGGCAGAAGTAAGGAGCAGGA,1
chr1,859052,860562,SAMD11,alpha_1,Islet1-fresh_AGACACCTATGCGCAGCGTCTAATGGTTG,1
chr1,859052,860562,SAMD11,alpha_2,Islet1-fresh_AGACACCTCGAGGCTGAAGGCTATGGTTG,1
chr1,859052,860562,SAMD11,alpha_2,Islet1-fresh_AGACACCTCGTACTAGCTAAGCCTGTACT,1
chr1,859052,860562,SAMD11,alpha_1,Islet1-fresh_AGACACCTGGACTCCTTCGACTAGGGTTG,1
chr1,859052,860562,SAMD11,alpha_2,Islet1-fresh_AGACACCTGGAGCTACAAGGAGTAAGGCG,1


 Concepts: 
1. `promoter region`: -500bp + 500bp for all TSS in gencode 
2. `promoter Peaks`: peaks that overlap promoter region

In [4]:
# get all cells 
dat.all.cells <- table((fread('../dat/output.umap.ab.filtered.csv'))$cluster) # table for all cells
dat.all.cells
sum(dat.all.cells)
length(unique(dat.mat$cell))


alpha_1 alpha_2  beta_1  beta_2 
   4266    1328    4354    2816 

[1] 12764

[1] 12764

## 2. Fisher's exact test at transcript level 

As long as there is any promoter peaks in that cell, the gene's promoter is open. 

1. get total alpha 1 and alpha 2 cells 
2. test hit in alpha1 vs hit in alpha2  (create contentigen table)
3. perform [Fisher's exact test](https://en.wikipedia.org/wiki/Fisher%27s_exact_test) or [chi-squared test](https://en.wikipedia.org/wiki/Chi-squared_test)

### 2.1 Prepare data

In [5]:
# prepare data
dat.mat.transcript_level <- dat.mat%>%
    select(-one_of("seq","start","end"))%>%
    distinct()%>%
    select(-cell)
dim(dat.mat.transcript_level)
head(dat.mat.transcript_level)

[1] 19412387        3

gene,cluster,transcript.idx
SAMD11,alpha_1,1
SAMD11,alpha_1,1
SAMD11,alpha_2,1
SAMD11,alpha_2,1
SAMD11,alpha_1,1
SAMD11,alpha_2,1


In [7]:
cat("Check how rows changed:\n") 
cat(sprintf("Before applying uniquness, # of rows:%d\n",nrow(dat.mat)))
cat(sprintf("After applying uniquness, # of rows:%d\n",nrow(dat.mat.transcript_level)))


Check how rows changed:
Before applying uniquness, # of rows:19412387
After applying uniquness, # of rows:19412387


### 2.2 perform fisher's exact test for all transcripts

In [8]:
cat(sprintf("There are %d unique transcripts for alpha cells\n",
            length(unique((dat.mat.transcript_level%>%filter(cluster %in%c("alpha_1","alpha_2")))$transcript.idx))))
cat(sprintf("There are %d unique transcripts for beta cells\n",
            length(unique((dat.mat.transcript_level%>%filter(cluster %in%c("beta_1","beta_2")))$transcript.idx))))

There are 20215 unique transcripts for alpha cells
There are 20336 unique transcripts for beta cells


#### 2.2.1 Special cases: TSS too close to two genes

- In this cases, put the same results to these genes.
- Note: `foverlap` results selected the 1st matches. 

In [10]:
#fun.ftestPerGene(dat=dat.sub,tr = x)
x <- 4016
head(dat.mat.transcript_level%>% filter(transcript.idx==x))

## sed -n 4015,4017p alpha.transcript_promoter_peaks.bed
cat("chr11	105947325	105948873	AASDHPPT\n
chr11	105947325	105948873	KBTBD3")

gene,cluster,transcript.idx
AASDHPPT,alpha_1,4016
KBTBD3,alpha_1,4016
AASDHPPT,alpha_1,4016
KBTBD3,alpha_1,4016
AASDHPPT,alpha_2,4016
KBTBD3,alpha_2,4016


chr11	105947325	105948873	AASDHPPT

chr11	105947325	105948873	KBTBD3

In [71]:
# all examples 
dat.sub.red <- dat.sub %>% select(-cluster)%>%group_by(transcript.idx)%>% unique()
setDT(dat.sub.red)
idx <- which(duplicated(dat.sub.red,by = "transcript.idx"))
head(dat.sub.red[idx,])
head(dat.sub.red[idx-1,])

gene,transcript.idx
SDF4,15
PUSL1,22
GLTPD1,24
RP4-758J18.2,31
SSU72,41
RER1,66


gene,transcript.idx
B3GALT6,15
ACAP3,22
CPSF3L,24
CCNL2,31
AL645728.1,41
MORN1,66


#### 2.2.2 Handle these special cases by cat gene and tr.idx

In [11]:
dat.mat.transcript_level<-dat.mat.transcript_level%>%
    unite("gene_tr.idx",c("gene","transcript.idx"),remove = T)
head(dat.mat.transcript_level)

gene_tr.idx,cluster
SAMD11_1,alpha_1
SAMD11_1,alpha_1
SAMD11_1,alpha_2
SAMD11_1,alpha_2
SAMD11_1,alpha_1
SAMD11_1,alpha_2


In [16]:
tr='SAMD11_1'
test.dat <- table((dat.sub%>% filter(gene_tr.idx==tr))$cluster)
table.res <- as.vector(test.dat)
table.res
names(test.dat)

[1] 83 47

[1] "alpha_1" "alpha_2"

In [17]:
fun.ftestPerTr <- function(  tr='SAMD11_1',#=1
                             celltypes=c('alpha_1','alpha_2'),
                             dat=dat.mat.transcript_level%>%
                                  select(one_of("gene_tr.idx","cluster")%>%
                                  filter(cluster %in%celltypes))){
    

    test.dat <- table((dat.sub%>% filter(gene_tr.idx==tr))$cluster)
    table.res <- as.vector(test.dat)
    names(table.res) <- names(test.dat)
    # handle if 0 for one subtype 
    a=setdiff(celltypes,    names(table.res))
    table.res[a]<-0
    
    test.tab <- matrix(c(table.res[celltypes[1]], table.res[celltypes[2]], 
                         dat.all.cells[celltypes[1]]-table.res[celltypes[1]], 
                         dat.all.cells[celltypes[2]]-table.res[celltypes[2]]),
                       byrow =  T,       
                       nrow = 2,
                       dimnames = list(expressed = c("Yes", "No"),
                       subtype = celltypes))
    test.res <- fisher.test(test.tab)
    res <- list(pval=test.res$p.value/2,
                odds=test.res$estimate,
                type1_frac=test.tab[1]/(test.tab[1]+test.tab[2]),
                type2_frac=test.tab[3]/(test.tab[3]+test.tab[4])
               )
    res
}

##

celltypes <- c('alpha_1','alpha_2')
dat.sub <- dat.mat.transcript_level%>%
                           select(one_of("gene_tr.idx","cluster"))%>%
                           filter(cluster %in%celltypes)
#fun.ftestPerGene(dat = dat.sub,tr=1)
system.time(fun.ftestPerTr(celltypes=celltypes))
fun.ftestPerTr(celltypes=celltypes)
system.time(fun.ftestPerTr(dat=dat.sub,celltypes=celltypes))  
fun.ftestPerTr(dat=dat.sub,celltypes=celltypes)
#all.tr <- unique(dat.sub$transcript.idx)
#for(x in all.tr){
#    fun.ftestPerGene(dat=dat.sub,tr = x)
#}



#fun.ftestPerGene(dat = dat.sub,tr=1)

   user  system elapsed 
  0.064   0.002   0.066 

$pval
[1] 0.0008038729

$odds
odds ratio 
 0.5408936 

$type1_frac
[1] 0.01945617

$type2_frac
[1] 0.03539157

   user  system elapsed 
  0.048   0.001   0.050 

$pval
[1] 0.0008038729

$odds
odds ratio 
 0.5408936 

$type1_frac
[1] 0.01945617

$type2_frac
[1] 0.03539157

In [22]:
celltypes <- c('beta_1','beta_2')
dat.sub <- dat.mat.transcript_level%>%
                           select(one_of("gene_tr.idx","cluster"))%>%
                           filter(cluster %in%celltypes)
fun.ftestPerTr(celltypes=celltypes)
fun.ftestPerTr(dat=dat.sub,celltypes=celltypes)


$pval
[1] 0.4757446

$odds
odds ratio 
  1.012853 

$type1_frac
[1] 0.0413413

$type2_frac
[1] 0.04083807

$pval
[1] 0.4757446

$odds
odds ratio 
  1.012853 

$type1_frac
[1] 0.0413413

$type2_frac
[1] 0.04083807

In [27]:
celltypes<- list()
celltypes$alpha <- c('alpha_1','alpha_2')
celltypes$beta <- c('beta_1','beta_2')

# time consumming task
if(T){
    require(parallel)
    system.time(res.transcript_level <- lapply(celltypes,function(x){
        dat.sub <- dat.mat.transcript_level%>%select(one_of("gene_tr.idx","cluster"))%>%filter(cluster %in% x)
        all.tr <- unique(dat.sub$gene_tr.idx)
        return(mclapply(all.tr,function(trr) fun.ftestPerTr(dat=dat.sub,tr = trr,celltypes = x),mc.cores = 8))}))
}
        

     user    system   elapsed 
  768.478    58.274 10749.944 

In [23]:
res.transcript_level.2 <- res.transcript_level

In [20]:
# alpha
x <- celltypes$alpha
dat.sub <- dat.mat.transcript_level%>%select(one_of("gene_tr.idx","cluster"))%>%filter(cluster %in% x)
all.tr <- unique(dat.sub$gene_tr.idx)
res.genes_level.a.df <- do.call(rbind,res.transcript_level$alpha)
rownames(res.genes_level.a.df) <- all.tr
head(res.genes_level.a.df)
res.transcript_level$alpha <-res.genes_level.a.df 

,pval,odds,type1_frac,type2_frac
SAMD11_1,0.0008038729,0.5408936,0.01945617,0.03539157
SAMD11_2,0.102674,0.6931602,0.008907642,0.0128012
SAMD11_3,0.000387331,0.4795206,0.01429911,0.02936747
NOC2L_4,0.1934535,0.8726655,0.0403188,0.04593373
KLHL17_5,0.2498725,0.8278814,0.01312705,0.01581325
PLEKHN1_6,0.000104741,0.43971,0.01312705,0.02936747


In [30]:
# beta
x <- celltypes$beta
dat.sub <- dat.mat.transcript_level%>%select(one_of("gene_tr.idx","cluster"))%>%filter(cluster %in% x)
all.tr <- unique(dat.sub$gene_tr.idx)
res.genes_level.a.df <- do.call(rbind,res.transcript_level$beta)
rownames(res.genes_level.a.df) <- all.tr
head(res.genes_level.a.df)
res.transcript_level$beta <-res.genes_level.a.df 

,pval,odds,type1_frac,type2_frac
SAMD11_1,0.4757446,1.012853,0.0413413,0.04083807
SAMD11_2,0.01671483,1.805673,0.01148369,0.006392045
SAMD11_3,0.002476792,1.409152,0.05029858,0.03622159
SAMD11_4,9.622286e-08,1.45374,0.1566376,0.1132812
NOC2L_5,1.421091e-10,1.948879,0.07854846,0.04190341
NOC2L_6,0.3060026,0.9384924,0.03674782,0.0390625


In [33]:
# adjust for p value 
m<- c("fdr","bonferroni","BY");names(m)<-c("FDR","padj.Bonferroni","FDR.BY")
res.transcript_level.2 <- lapply(res.transcript_level,function(df){
    res.genes_level.a.df <- as.data.frame(df)
    for(i in 1:3){
        res.genes_level.a.df[[names(m)[i]]] <- p.adjust(as.numeric(res.genes_level.a.df$pval),method = m[i])
    }
    res.genes_level.a.df$odds <- as.numeric(res.genes_level.a.df$odds)
    res.genes_level.a.df
})
head(res.transcript_level$alpha)
head(res.transcript_level.2$beta)

,pval,odds,type1_frac,type2_frac,FDR,padj.Bonferroni,FDR.BY
SAMD11_1,0.0008038729,0.5408936,0.01945617,0.03539157,0.0027715079,1,0.029346494
SAMD11_2,0.102674,0.6931602,0.008907642,0.0128012,0.1551145157,1,1.000000000
SAMD11_3,0.000387331,0.4795206,0.01429911,0.02936747,0.0014961208,1,0.015841881
NOC2L_4,0.1934535,0.8726655,0.0403188,0.04593373,0.2535538133,1,1.000000000
KLHL17_5,0.2498725,0.8278814,0.01312705,0.01581325,0.3080746607,1,1.000000000
PLEKHN1_6,0.000104741,0.4397100,0.01312705,0.02936747,0.0004890037,1,0.005177883


,pval,odds,type1_frac,type2_frac,FDR,padj.Bonferroni,FDR.BY
SAMD11_1,0.4757446,1.0128525,0.0413413,0.04083807,4.913540e-01,1.000000e+00,1.000000e+00
SAMD11_2,0.01671483,1.8056728,0.01148369,0.006392045,4.398008e-02,1.000000e+00,4.658901e-01
SAMD11_3,0.002476792,1.4091523,0.05029858,0.03622159,1.038851e-02,1.000000e+00,1.100476e-01
SAMD11_4,9.622286e-08,1.4537397,0.1566376,0.1132812,3.228731e-06,2.153564e-03,3.420262e-05
NOC2L_5,1.421091e-10,1.9488789,0.07854846,0.04190341,1.314274e-08,3.180543e-06,1.392237e-07
NOC2L_6,0.3060026,0.9384924,0.03674782,0.0390625,3.669637e-01,1.000000e+00,1.000000e+00


In [40]:
res.transcript_level <- res.transcript_level.2
saveRDS(res.transcript_level,"../dat/1901/res.transcript_level.rds")